##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Génération de texte avec un RNN

Ce tutoriel explique comment générer du texte à l'aide d'un RNN basé sur des caractères. Nous allons travailler avec un 
ensemble de données de l'écriture de Molière tirée de Project Gutenberg (http://www.gutenberg.org). Étant donné une séquence de 
caractères, entraînez un modèle pour prédire le prochain caractère de la séquence. Par exemple, si nous sommes donnés la 
séquence de caractères "Molièr", prédire le prochain caractère tel que "e". Des séquences de texte plus longues peuvent être 
générées en appelant le modèle à plusieurs reprises.

Activez l'accélération GPU pour exécuter ce notebook plus rapidement. Dans Colab: *Runtime > Change runtime type > Hardware 
acclerator > GPU*.

Ce tutoriel comprend du code de [tf.keras] (https://www.tensorflow.org/programmers_guide/keras) et de [eager execution] 
(https://www.tensorflow.org/programmers_guide/eager). Voici un exemple de sortie lorsque le modèle de ce tutoriel a été 
entraîné pour 30 époques et a démarré avec la chaîne de caractères "Q":

<pre>
    DON PÈDRE.

    Il est vrai, ma fille, vous représentez un peu la conscience,
      Gardez qu'on accommoder sur ce moment comme lui.

        SOSIE.

                  Me voilà donc, monsieur, d'une âme:
        Tous deux également tendent toute l'amour que cela pourra done.

    SCÈNE III.--DON JUAN, SGANARELLE.

    DON JUAN.

    Me fera-t-on l'âme te fait? J'ai voulu se tacevoir tout nu, puisque tu puis voir accepter votre petits vertus,
    Qui me fait souffrir qu'un tour exemplaisir;
      Et ce choix démentir maître un certe montsir Oronte, Dupa chose;
    Et, tandis qu'à cause les plus grands médecins tous les diables. Et mous offrir médecins et indivertirs son
    nennence de sa façon.

  MARPHURIUS.

  Assurément; mais mon cher marquis, je ne l'équez vous plaire;
    Ces grands satyres affreux des chanses le plus vite que vous avez
    que je me sers avec grands merveilles; et c'est là votre personne en mêle,
    Des lui soient prévenus de la beauté du ciel;
     Dander de la belle parole; je suis assez humani.
    Je serai cette prière raison d'être médecine?

    SGANARELLE.

    Ce vie embarrasser qu'on n
</pre>

Bien que certaines phrases soient grammaticales, la plupart n’ont pas de sens. Le modèle n'a pas appris le sens des mots, mais 
considérez:

* Le modèle est basé sur les caractères. Lorsque la formation a commencé, le modèle ne savait pas comment épeler un mot 
français, ni que ces mots constituaient même une unité de texte.

* La structure de la sortie ressemble aux blocs de texte dans une pièce commencent généralement par un nom de locuteur, en 
toutes lettres majuscules, similaire à l'ensemble de données.

* Comme illustré ci-dessous, le modèle est entraîné sur les petites séquences de texte (100 caractères chacun), mais est 
capable de générer une séquence de texte plus longue avec une structure cohérente.

## Préparatifs

### Importer TensorFlow et d'autres bibliotheques (libraries)

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
#from google.colab import files
#!mkdir data
#%cd data
#upload = files.upload()
#%cd ..

try:
  tf.enable_eager_execution()
except Exception:
  pass

### Telecharger les données de Shakespeare

Vous pouvez modifier la ligne suivante pour exécuter ce code sur vos propres données.

In [4]:
# lire le fichier de l'internet, utilisez le code suivant si vous avez un version local du fichier
# path_to_file = tf.keras.utils.get_file('molière.txt', 'https://gdurl.com/VYKa')

# lire le fichier local
path_to_file = 'data/molière50000.txt'

### Lire les données

Jetez un coup d'œil sur le texte

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1443686 characters


In [7]:
# Regardez les premièrs 500 caractères dans le texte
print(text[:500])

    VALÈRE.

                  Mais quoi! l'honneur de vous connoître
    Est un si grand bonheur, est un si doux plaisir,
    Que de vous saluer j'avois un grand désir.

    SGANARELLE.

    Soit.

    VALÈRE.

          Et de vous venir, mais sans nul artifice,
    Assurer que je suis tout à votre service.

    SGANARELLE.

    Je le crois.

    VALÈRE.

                 J'ai le bien d'être de vos voisins,
    Et j'en dois rendre grâce à mes heureux destins.

    SGANAR


**Tâche 1 (Caractères Features)** Selectionnez et triez les features (caractères uniques).

In [8]:
# trouvez tous les caractères uniques dans le texte
vocab = sorted(set(text))
# vocab = #######  Votre code ici #######
print ('{} unique characters'.format(len(vocab)))

93 unique characters


## Traiter le texte

### Vectoriser le texte

Afin de préparer pour l'étape d'entraînement, nous devons convertir les chaînes de caractères à une représentation numérique. Créez deux lookup tables (table de correspondance): une mappant des caractères aux nombres et une autre pour des nombres aux caractères.

In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

**Tâche 2 (Préparez le texte)**  Convertissez les caractères du texte en valeurs entières.

In [10]:
text_as_int = np.array([char2idx[c] for c in text])
# text_as_int = np.array( ####### Votre code ici ####### )

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '0' :  10,
  '1' :  11,
  '2' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '8' :  16,
  ':' :  17,
  ';' :  18,
  '?' :  19,
  ...
}


In [12]:
# Montrez comment les 13 premiers caractères du texte sont mappés à des valeurs entières
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))
# Montrez comment les 13 derniers caractères du texte sont mappés à des valeurs entières
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[-13:]), text_as_int[-13:]))

'    VALÈRE.\r\n' ---- characters mapped to int ---- > [ 2  2  2  2 40 20 30 73 36 24  9  1  0]
', SBRIGANI.\r\n' ---- characters mapped to int ---- > [ 7  2 37 21 36 28 26 20 32 28  9  1  0]


### La prévision par caractère

Étant donné un caractère ou une séquence de caractères, quel est le caractère suivant le plus probable? C'est la tâche que nous entraînerons le modèle à effectuer. L'entrée au modèle sera une séquence de caractères, et nous apprenons au modèle à prédire la sortie - le caractère suivant à chaque pas de temps.

Plus précisément pour le RNN, puisque les RNN maintiennent un état interne qui dépend des éléments vus précédemment, la tâche se propose comme : étant donné tous les caractères calculés jusqu'à ce moment, quel est le prochain caractère ?


### Créer des textes pour l'entraînement et ses cibles (les bonnes réponses)

Divisez le texte en séquences. Chaque séquence d'entrée contiendra les caractères `seq_length` du texte.

Pour chaque séquence d'entrée, les cibles correspondantes contiennent la même longueur de texte, à l'exception du caractère décalé d'un caractère vers la droite.

Donc, divisez le texte en morceaux de `seq_length + 1`. Par exemple, disons `seq_length` vaut 4 et notre texte est "Hello". La séquence d'entrée serait "Hell" et la séquence cible "ello".

D’abord, nous utiliserons la fonction `tf.data.Dataset.from_tensor_slices` pour convertir le vecteur de texte en un flux (stream) d’index de caractères.

**Tâche 3 (Datasets)**

In [13]:
# **seq_length** La longueur maximale en caractères de la phrase pour une seule entrée
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Créez morceaux de texte pour l'entraînement et les cibles
# char_dataset = ### VOTRE CODE ICI Créez le Dataset char_dataset des tensor slices (le texte vectorisé) ###
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

print('Eager execution enabled?  ' + str(tf.executing_eagerly()))

# .Numpy peut être utiliser uniquement en mode eager. En mode graphique, vous devez utiliser
# eval dans une session pour obtenir la valeur du tenseur dans le tableau numpy.
if tf.executing_eagerly() :
#    for i ### VOTRE CODE ICI Imprimez les premiers 5 éléments du Dataset pour l'entraînement ###
    for i in char_dataset.take(5):        
        print(idx2char[i.numpy()])
else :
    iter = char_dataset.make_one_shot_iterator()
    with tf.Session() as sess:
        for x in range(0,5):
            print(idx2char[(sess.run(iter.get_next()))])

Eager execution enabled?  True
 
 
 
 
V


La méthode `batch` combine des éléments consécutifs d'un Dataset en lots (batch). Cela nous permet de convertir facilement des caractères individuels en séquences de la taille souhaitée **seq_length+1**.

In [14]:
# sequences = ### VOTRE CODE ICI Combine des éléments (batch) du Dataset d'entraînement. Utilisez la taille seq_length+1 et
# assurez que tous les batch (le dernier compris) ont la même taille. ###
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# for item ### VOTRE CODE ICI Imprimez les premières 5 séquences ###
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"    VALÈRE.\r\n\r\n                  Mais quoi! l'honneur de vous connoître\r\n    Est un si grand bonheur,"
" est un si doux plaisir,\r\n    Que de vous saluer j'avois un grand désir.\r\n\r\n    SGANARELLE.\r\n\r\n    So"
'it.\r\n\r\n    VALÈRE.\r\n\r\n          Et de vous venir, mais sans nul artifice,\r\n    Assurer que je suis to'
"ut à votre service.\r\n\r\n    SGANARELLE.\r\n\r\n    Je le crois.\r\n\r\n    VALÈRE.\r\n\r\n                 J'ai le"
" bien d'être de vos voisins,\r\n    Et j'en dois rendre grâce à mes heureux destins.\r\n\r\n    SGANARELLE."


**map** exécute des fonctions sur les éléments de le dataset et renvoie un nouvel dataset contenant les éléments transformés, dans le même ordre de l'entrée.
Pour chaque séquence, dupliquez-la et décalez-la d'un caractère pour créer le texte d'entrée et le texte cible en utilisant la méthode `map` pour appliquer l'objet split_input_target aux séquences déjà découpées en tranches.

**Tâche 3** Créez le texte d'entrée et son texte cible (la bonne réponse), les données d'entraînement (train).

In [15]:
def split_input_target(chunk):
#    input_text = ####### Votre code ici #######
#    target_text = ####### Votre code ici #######
    input_text = chunk[:-1]
    target_text = chunk[1:]    
    return input_text, target_text

# dataset = ### VOTRE CODE ICI Utilisez le method map pour exécuter la fonction split_input_target afin de transformer 
# les séquences en nouvelles séquences décaler par un caractère
dataset = sequences.map(split_input_target)

**Tâche 4 Compréhension**
Imprimer la première séquence : le texte d'entrée et le texte ciblé. Assurez-vous que vous comprenez au-dessous comment la fonction split_input_target est exécutée sur le premier élément du Dataset dataset afin de produire les deux chaînes de caractères.

In [16]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  "    VALÈRE.\r\n\r\n                  Mais quoi! l'honneur de vous connoître\r\n    Est un si grand bonheur"
Target data: "   VALÈRE.\r\n\r\n                  Mais quoi! l'honneur de vous connoître\r\n    Est un si grand bonheur,"


Chaque elément (@index) de ces vecteurs est traité comme un pas de temps. Pour l'entrée au pas de temps 0, le modèle reçoit l'index pour la toute première lettre "F" et essaie de prédire l'indice pour "i" en tant que caractère suivant. Au pas de temps suivant, il fait la même chose mais le `RNN` considère le contexte de l’étape précédente en plus du caractère d’entrée actuel.

Nous imprimons les premiers 5 étapes avec la lettre d'entrée (pas de temp n) et la lettre prédite (pas de temp n+1)

In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 2 (' ')
  expected output: 2 (' ')
Step    1
  input: 2 (' ')
  expected output: 2 (' ')
Step    2
  input: 2 (' ')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 40 ('V')
Step    4
  input: 40 ('V')
  expected output: 20 ('A')


### Créer des lots (batch) d'entraînement

Utiliser `tf.data` pour scinder le texte en séquences gérables (batch). La taille des lots (BATCH_SIZE) est une paramètre relative à la performance, typiquement reposant sur l'expérience. Puis, nous allons mélanger (shuffle) les lots. En mélangeant les lots, on essaye de rendre l'entraînement plus robuste.

**Tâche 5** Scindez (batch), puis, mélangez (shuffle) les lots. Utilisez les methodes de tf.data.Dataset. Fixez la paramètre
drop_remainder=True.

In [18]:
# Taille de lots en octets
BATCH_SIZE = 64

# Taille du tampon utilisé pour mélanger les lots en octets
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# dataset = ####### VOTRE CODE ICI #######

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## Construire le modèle

**Tâche 6** Ecrivez le code pour le fonction build_model en bas.
Utilisez `tf.keras.Sequential` pour définir le modèle. Pour notre projet, trois couches (layers) sont utilisées pour définir notre modèle:

* `tf.keras.layers.Embedding`: La couche d'entrée. Une table de consultation (lookup table) utiliser pour mapper les numéros de chaque caractère à un vecteur de dimensions `embedding_dim`;
* `tf.keras.layers.GRU`: Un RNN de taille `units=rnn_units`
* `tf.keras.layers.Dense`: La couche de sortie, avec `vocab_size` pour la dimension de la sortie .
Les astuces sont données en bas pour configuer les paramètres pour chaque couche.

Pour chaque caractère, le modèle trouve l'embedding, exécute GRU un seul pas de temps avec l'embedding en tant qu'entrée et applique la couche dense pour générer des logits (numéro réel) prédisant le log-likelihood (probabilité) du caractère suivant.

In [19]:
# Taille du vocabulaire en caractères
vocab_size = len(vocab)

# La dimension de la couche d'entrée Embedding.
embedding_dim = 256

# Nombre d'unités RNN
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model
#def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#    # Taille des séquences d'entrée
#  Embedding_input_length = [batch_size, None]    
#  # astuces paramétres Embedding layer
#  #    Fixer la valeur de vocab_size, embedding_dim, et batch_input_shape=[batch_size, None]
#  #    ne pas utiliser: embeddings_initializer, embeddings_regularizer, embeddings_constraint, mask_zero
#  # astuces paramétres GRU Layer
#  #    Fixez la valuer de rnn_units. Fixez return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'
#  #    ne pas utiliser tous autres paramétres
#  # astuces paramétres Dense Layer
#  #    Fixez la valeur de units. Ne pas utiliser tous autres paramétres.
#    model = ######## VOTRE CODE ICI #######
#  return model

In [21]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Essayer le modèle (avant l'entraînement)

Le modèle peut être utilisé directement sans entraînement. On ne peut pas attendre les bons résultats, mais nous pouvons vérifier que le fonctionnement du modèle.

La taille de séquence de l'entrée est 100 mais le modèle peut être exécuté sur des entrées de n'importe quelle taille.

Nous exécutons le model sur la première séquence dans le Dataset (qui est aléatoire parce que nous avons effectué un "shuffle". Puis, nous imprimons le shape (dimensions) de la sortie (les prévisions). (Pourquoi ces dimensions?)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 93) # (batch_size, sequence_length, vocab_size)


Maintenant, nous regardons à la structure du modèle. Nous voyons les dimensions de chaque couche (pourquoi ces dimensions?) et le nombre de paramètres entraînable dans chaque couche (ce numéro dépend sur la composition interne de la couche).

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23808     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 93)            95325     
Total params: 4,054,365
Trainable params: 4,054,365
Non-trainable params: 0
_________________________________________________________________


**example_batch_predictions** contient les probabilités (logits) initiales pour la chaîne de 100 caractères (_sequence_length_) dans les 64 chaînes (_batch_size_). Nous allons trouver les indices de caractères pour chaque prévision.

Nous prenons les échantillons aléatoires (_random.categorical_). Ils sont les valeurs initiales pour démarrer l'entraînement.  Les valeurs sont trouvées aléatoirement et pas simplement par le _argmax_ de chaque lot (batch) afin que l'entraînement progresse bien vers un vrai minima (en raison de le comportement de gradient descent).

Nous l'essayons pour le premier exemple dans les lots.

In [24]:
print(len(example_batch_predictions))
print(len(example_batch_predictions[0]))
print(example_batch_predictions)

64
100
tf.Tensor(
[[[ 6.91796700e-03 -4.81686369e-03  5.80169354e-03 ...  8.91180534e-04
    8.71576369e-03  1.14737367e-02]
  [ 1.06651243e-02 -8.04523937e-03  9.07366443e-03 ... -1.00196723e-03
    1.14420317e-02  1.76288877e-02]
  [-9.69880261e-03 -6.50260691e-03  6.77416334e-03 ...  4.16109571e-03
    6.94275601e-03  2.18349304e-02]
  ...
  [-5.99234505e-03 -5.33073861e-03 -5.11327712e-03 ... -1.20772058e-02
    2.51080521e-04 -6.74794428e-03]
  [-1.77461486e-02 -3.91830737e-03  4.56304289e-03 ... -4.01284196e-04
   -3.16732959e-03  6.87833689e-03]
  [-4.72973660e-03 -7.25232624e-03 -1.94557942e-05 ... -1.92286179e-03
   -3.02941143e-03  9.66562517e-03]]

 [[ 3.02027632e-03 -8.24951334e-04 -1.67923905e-02 ... -2.05681706e-03
    5.09790704e-03 -3.29531846e-03]
  [-7.90944044e-03  5.16581442e-03 -1.90199241e-02 ...  9.82866809e-03
    1.13448342e-02  4.45548352e-03]
  [-5.11800218e-03 -5.49809542e-03 -1.46676805e-02 ...  3.67439282e-03
    1.21212620e-02  2.16381531e-03]
  ...
  [ 7

In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [26]:
print(len(sampled_indices))
print(sampled_indices)

100
tf.Tensor(
[[75]
 [46]
 [37]
 [86]
 [68]
 [69]
 [24]
 [81]
 [92]
 [52]
 [36]
 [28]
 [ 0]
 [ 0]
 [82]
 [90]
 [ 3]
 [20]
 [50]
 [34]
 [29]
 [69]
 [92]
 [62]
 [17]
 [ 8]
 [ 8]
 [76]
 [22]
 [ 0]
 [ 9]
 [43]
 [89]
 [10]
 [19]
 [27]
 [71]
 [91]
 [ 8]
 [40]
 [ 9]
 [53]
 [10]
 [90]
 [62]
 [72]
 [20]
 [30]
 [68]
 [53]
 [54]
 [ 7]
 [ 9]
 [79]
 [28]
 [70]
 [37]
 [79]
 [40]
 [77]
 [47]
 [ 5]
 [46]
 [83]
 [60]
 [54]
 [21]
 [22]
 [35]
 [38]
 [12]
 [86]
 [57]
 [54]
 [11]
 [26]
 [20]
 [11]
 [81]
 [68]
 [35]
 [75]
 [21]
 [38]
 [48]
 [ 4]
 [26]
 [91]
 [30]
 [39]
 [30]
 [43]
 [15]
 [25]
 [ 3]
 [92]
 [18]
 [56]
 [20]
 [68]], shape=(100, 1), dtype=int64)


_squeeze_ pour faire une 1D _numpy_ tableau (array)

In [27]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

Maintenant, pour chaque pas de temp, nous avons une prévision pour le prochain indice de caractère.

In [28]:
sampled_indices

array([75, 46, 37, 86, 68, 69, 24, 81, 92, 52, 36, 28,  0,  0, 82, 90,  3,
       20, 50, 34, 29, 69, 92, 62, 17,  8,  8, 76, 22,  0,  9, 43, 89, 10,
       19, 27, 71, 91,  8, 40,  9, 53, 10, 90, 62, 72, 20, 30, 68, 53, 54,
        7,  9, 79, 28, 70, 37, 79, 40, 77, 47,  5, 46, 83, 60, 54, 21, 22,
       35, 38, 12, 86, 57, 54, 11, 26, 20, 11, 81, 68, 35, 75, 21, 38, 48,
        4, 26, 91, 30, 39, 30, 43, 15, 25,  3, 92, 18, 56, 20, 68],
      dtype=int64)

Et convertir l'indice en caractère pour voir le texte prévu.

In [29]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "llez, j'étois trop dupe, et je vais ne plus l'être;\r\n      Vous me faites un bien, me faisant vous c"

Next Char Predictions: 
 "ÊaSîyzEèœgRI\n\néù!AePJzœr:--ÏC\n.Zô0?H»û-V.h0ùrÇALyhi,.æI«SæVàb(aêpiBCQT2îmi1GA1èyQÊBTc'GûLULZ6F!œ;lAy"


## Entraîner le modèle

Maintenant, le problème est convenablement configuré et nous pouvons le traiter comme une classification normale. Donnant l'état précédente du RNN, et la donnée d'entrée pour le pas de temps actuel, prédire la classe de le prochain caractère.

### Definir une loss function (fonction de perte), Compiler le modèle avec un optimizer (optimisateur) et la loss fonction.

Nous utiliserons la loss function `tf.keras.losses.sparse_categorical_crossentropy`

In [30]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5331798


**Tâche 7** Compilez les paramètres de l'entraînement avec `tf.keras.Model.compile` . Nous emploierons juste deux paramètres, la méthode de réviser les parametres (optimizer) et la méthode pour calculer la fonction de perte (loss). L'optimizer recommandé est 'adam' (`tf.keras.optimizers.Adam`). L'optimisation d'Adam est une méthode de descente de gradient stochastique (stochastic gradient descent) basée sur une estimation adaptative des moments du premier et du second ordre. La fonction de perte est la fonction que vous avez créée et testée au-dessus. 

In [31]:
model.compile(optimizer='adam', loss=loss)
####### VOTRE CODE ICI #######

### Configuer les checkpoints (pointes de contrôles)

Nous allons sauvegarder les checkpoints (pointes de contrôle) pour chaque epoch avec les poids (weights) que le modèle a trouvé.

In [32]:
# Dosser où les checkpoints seront sauvegardés
checkpoint_dir = './training_checkpoints'
# Nom de chaque fichier checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Executer l'entraînement

Commençons avec 10 époques pour l'entraînement. Nous pouvons choisir d'utiliser le GPU si nous faisons l'entraînement sur Colab pour raccourcir le temps. Sur mon ordinateur avec un Ryzen 5 chaque époque a pris 20 minutes, sur le GPU de Colab, 24 secondes. Le CPU de Colab (sans accéleration) a pris 16 minutes.

In [33]:
EPOCHS=30

In [33]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
223/223 [==============================] - 1244s 6s/step - loss: 2.3382
Epoch 2/30
223/223 [==============================] - 1255s 6s/step - loss: 1.5338
Epoch 3/30
223/223 [==============================] - 1256s 6s/step - loss: 1.2865
Epoch 4/30
223/223 [==============================] - 1255s 6s/step - loss: 1.1564
Epoch 5/30
223/223 [==============================] - 1252s 6s/step - loss: 1.0825
Epoch 6/30
223/223 [==============================] - 1240s 6s/step - loss: 1.0342
Epoch 7/30
223/223 [==============================] - 1231s 6s/step - loss: 0.9969
Epoch 8/30
223/223 [==============================] - 1224s 5s/step - loss: 0.9665
Epoch 9/30
223/223 [==============================] - 1218s 5s/step - loss: 0.9412
Epoch 10/30
223/223 [==============================] - 1236s 6s/step - loss: 0.9154
Epoch 11/30
223/223 [==============================] - 1245s 6s/step - loss: 0.8929
Epoch 12/30
223/223 [==============================] - 1243s 6s/step - loss: 0.8712
E

## Génération du texte

### Restauer le dernier checkpoint (point de contrôle)

Pour l'inférence (utilisation d'un modèle entraîné pour les prévisions) nous pouvons traiter les données avec un taille de lot (_batch_size_) = 1. Mais, il faut reconstruire (_build_) le modèle avec la nouvelle _batch_size_. On a besoin juste de lui fournir les poids (_weights_) finals, pas besoin de répéter l'entraînement.

In [34]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23808     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 93)             95325     
Total params: 4,057,437
Trainable params: 4,057,437
Non-trainable params: 0
_________________________________________________________________


### La boucle de prévision

La boucle suivante de code générera le texte:

* Initialisation: une chaîne de caractères de début, l'état du RNN et le nombre de caractères à générer.

* Obtenez la distribution de prédiction du prochain caractère avec la chaîne de caractère de début et de l'état du RNN.

* Ensuite, utilisez la distribution catégorique pour calculer l’indice du caractère prédit. Puis, utilisez ce caractère prédit comme la prochaine entrée dans le modèle.

* L'état du RNN renvoyé par le modèle rentre encore dans le modèle afin qu'il ait maintenant plus de contexte, au lieu d'une seule caractère. Après avoir prédit le mot suivant, les états des RNN modifiés sont à nouveau rentrés dans le modèle, ainsi le modèle apprend au fur et à mesure qu'il récupère le contexte des mots prédits précédemment.

En regardant le texte généré, vous verrez que le modèle sait faire les lettres majuscules, faire des paragraphes et imiter un vocabulaire d'écriture semblable à celui de Shakespeare. Avec le petit nombre d’époques de formation, il n’a pas encore appris à former des phrases cohérentes.

In [0]:
def generate_text(model, start_string):
# L'étape de l'evaluation (génération du text par un modèle entraîné)

  # Nombre de caractères à générer
  num_generate = 1200

  # Convertir la chaîne de caractères de début aux nombres (vectorisation)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Chaîne de caractères pour le résultat
  text_generated = []

  # Les températures basses donnent un texte plus prévisible.
  # Des températures plus élevées produisent un texte plus surprenant.
  # Faites des essais pour trouver le meilleur réglage.
  temperature = 1.0

  # batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # enlever une dimension batch (squeeze)
      predictions = tf.squeeze(predictions, 0)

      # on utilise la distribution catagorique pour prédire le texte renvoyé par le modèle
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # la lettre prédit est ajouté à la prochaine entrée du modèle  
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [43]:
print(generate_text(model, start_string=u"S"))

SCÈNE I.--CLITANDRE, ALCESTE.

      CLÉANTE, à Mariane.

      Oui; et j'attendois de votre ennui?
      Et quel prince s'os languis j'aimerois bien vivre
    Pour abis d'achever sur vous, ma biensi.

      GEORGE DANDIN, à madame de
    sa chère chez nous.

      GEORGE DANDIN.

      Eh! Ah! quelle
  consulter ici.

    LÉANDRE.

    Monsieur, vont ai-je lieu de la  dessein étrange partie de votre mariage?

    ARNOLPHE.

                             Las!

      SCÈNE V.--Il a souffert une comédie à vous voir dehors pour vous dise? On gréondre, je suis boi et mon amour dont je n'ai point langé de l'aMIERLOTTE, FRASTE, LA MONTAGNE.

    ÉRASTE.

    Il n'a des tiens d'ici, tu es cruelle,
      Et qu'à moi, quelque espoir à tout cet espoir pourtant, emment comme de le déshonorer de ce que je sens; et,
    suis-je savoir d'un amant à Molière, que vous voyez, croyant virge le mieux du monde,
    Madame, enfin lans ce doit être vos personnages,
    Et je l'aime pour patience.

      LUBI

**ESSAYER D'AMELIORER VOS RESULTATS**
La méthode la plus facile à améliorer les résultats est d'augmenter le nombre d'époques d'entraînement (essayez `EPOCHS=30`).

Vous pouvez également expérimenter avec une autre chaîne de caractères de départ ou essayer d'ajouter une autre couche RNN pour améliorer la précision du modèle ou d'ajuster le paramètre de température pour générer des prédictions plus ou moins aléatoires.